# redis基础知识

要知其所以然

## 简介

遵守BSD协议 ?

高性能key-value数据库

与其他key-value缓存产品相比的特点：

1. 支持持久化，内存-》硬盘；重启后 内存 《-硬盘
2. 数据类型 key-value；list；set；zset；hash；
3. 支持数据备份，即master-slave模式的数据备份；


## 优势

1. 速度 读11w/s；写8.1w/s
2. 支持二进制的Strings,List,Hashes,Sets,Ordered Sets数据类型操作
3. 原子性。事务。
4. 至此 pub/sub；通知；key过期；


## 与其他key-value存储的不同

1. 基于基本数据结构，对程序员透明
2. 在内存中操作数据简单，所以飘了；磁盘格式方面是追加的方式，不需要随机访问；

# 配置
config [get|set] name
扁平化配置

## 数据类型
支持5种数据类型：string,hash,list,set,zset(有序集合)

### String

1. 最基本类型，同Memcached,key-value
2. 二进制安全，可以包含任何数据jpg，序列化；
3. < 512MB

例：
```
    SET name "block"
    Get name
```

### Hash

1. [key-value...] 对的集合
2. hash是string类型的field和value的映射表，hash特别适合存储对象

例：
```
    HMSET myhash field1 "hello" field2
    HGET myhash field1
    HGET myhash field2
```

### List

1. 的字符串列表，按照插入顺序排序。可以向列表头部，或尾部添加；
2. 最多 2^32-1个元素（40亿+）
例：
```
    lpush runoob redis
    lpush runoob monogodb
    
    lrange runoob 0 10
```

### Set

1. string类型的无序集合
2. 集合是通过hash表实现的，增删查的复杂度为O（1）

例：
```
    sadd runnoob redis
    sadd runnoob monogodb
    
    smember runoob
```
### zset(sort set)

1. 不重复，因为每个元素都会关联一个double类型的分数。类比hash排序；
2. 增加元素到集合，更新对应的score

例：
```
    zadd key score member
    zadd runoob 0 redis
    zadd runoob 0 monogodb
    
    ZRANGEBYSOCRE runoob 0 1000
```


## HyperLogLog
基数估计,集合中不重复元素个数

例：
```
    PFADD runoobkey "redis"
    PFADD runoobkey "redis2"
    
    PFCOUNT runoobkey    
```

## 发布订阅
一种通过channel实现的发布订阅模式

例：
```
    SUBSCRIBE redisChat
    
    PUBLISH redisChat "Learn redis by runoob.com"
```



## 事务

1. 批量操作在发送exec命令前被放入队列缓存
2. 收到exec命令后，进入事务执行，任意命令失败，其余命令依然会被执行
3. 在事务执行过程中，其他客户端提交的命令请求不会插入到事务执行命令序列中

开始事务-命令入队-执行事务

例：
```
    MULTI
    SET book-name
    SADD tag "go" "program" "routine"
    SMEMBERS tag
    EXEC
```

redis的事务只是把命令打包了，不是原子性的，不会回滚，也不会提前终止


## 脚本

使用Lua解释器来执行脚本

`EVAL script numkeys key [key ...] arg [arg ...]`

例：
```
EVAL "return {KEYS[1],KEYS[2],ARGV[1],ARGV[2]}" 2 key1 key2 first second
```


## 连接

AUTH "password"

PING